In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data = pd.read_csv('./Data/scaled_grad_demo_fin.csv')

In [3]:
data

,grad_rate,ECONOMICALLY_DISADVANTAGED,H_Female,AA_FEMALE,W_FEMALE,AA_MALE,W_MALE,H_MALE,AA,H,W,Expend_per_pupil
0,0.647137,-0.340444,-0.810652,-0.749432,0.881421,-0.731183,0.939311,-1.003977,-0.743636,-0.922150,0.926043,-0.247486
1,0.496101,0.295084,-0.742734,-0.610572,0.488740,-0.632984,0.972940,-0.629554,-0.624308,-0.691083,0.747043,-0.077559
2,0.042992,-0.669709,0.225286,-0.220271,0.073510,-0.104159,-0.119980,0.183801,-0.163735,0.205828,-0.025222,1.379947
3,0.949210,-0.714760,0.031734,-0.641120,0.294973,-0.573553,0.814632,-0.067018,-0.610439,-0.020230,0.568360,-0.278540
4,-0.410118,0.520390,3.856570,-0.095779,-0.799500,-0.123050,-0.743684,3.180804,-0.109695,3.541708,-0.784013,-1.092946
...,...,...,...,...,...,...,...,...,...,...,...,...
262,0.345064,-0.340087,0.645130,-0.160112,-0.038207,-0.149754,-0.012516,0.629353,-0.155676,0.643992,-0.025573,-0.888277
263,0.798174,-1.560176,-0.530941,-0.422304,0.470175,-0.385032,0.388020,-0.400242,-0.405676,-0.467638,0.435583,-1.044175
264,0.647137,-0.768097,-0.901072,-0.606816,0.650322,-0.509172,0.773584,-0.499283,-0.561095,-0.698286,0.724858,0.325878
265,0.798174,-1.298110,-0.259168,-0.374950,0.246499,-0.412574,0.458281,-0.167937,-0.395199,-0.213734,0.360024,-0.814397


In [4]:
X = data.drop(columns=['grad_rate'])
y = data[['grad_rate']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state = 40)

In [6]:
MLR_model = LinearRegression()
MLR_model.fit(X_train, y_train)

LinearRegression()

In [7]:
MLR_model.score(X_train, y_train)

0.5870716921652415

In [8]:
y_pred = MLR_model.predict(X_test)

In [9]:
mean_squared_error(y_test, y_pred)

0.6034957271086209